In [1]:
import olpy
import openlattice
import pandas as pd
import sqlalchemy as sq
import re
import pytz
import numpy as np
import os

In [2]:
conf = openlattice.Configuration()
conf.host = "https://api.openlattice.com"
conf.access_token = "<INSERT JWT TOKEN>"

entitySetsAPI = openlattice.EntitySetsApi(openlattice.ApiClient(conf))
dataAPI = openlattice.DataApi(openlattice.ApiClient(conf))
dataSetMetadataAPI = openlattice.DataSetMetadataApi(openlattice.ApiClient(conf))

## To get a metadata for a dataset or a column

First find the UUID for the dataset or column. To find the UUID for the dataset, either check out the [orgs page](https://openlattice.com/orgs/#/orgs/) and find the table you are looking for. The UUID will be the last part of the URL. Alternatively, use `get_organization_data_sets_metadata` and input the organization ID as the argument.

Once you find the data set metadata, then use `get_data_set_metadata` or `get_data_set_column_metadata` to get the appropriate metadata result. Pass in the UUID of the data set for `get_data_set_metadata` or the UUID of the data set and UUID of the column to for `get_data_set_column_metadata`.

In [3]:
dataSetMetadataAPI.get_organization_data_sets_metadata("92161827-b871-4ab5-8857-c895e7fa863c")

{'c681fe24-e493-4bfd-b427-310c8b430672': {'data_set_type': 'EntitySet',
  'external_id': 'c681fe24-e493-4bfd-b427-310c8b430672',
  'id': 'c681fe24-e493-4bfd-b427-310c8b430672',
  'metadata': {'contacts': [],
               'description': 'This is an automatically generated auditing '
                              'entity set.',
               'flags': ['AUDIT', 'ASSOCIATION'],
               'metadata': [],
               'title': 'Audit edge entity set for '
                        'org-metadata-92161827-b871-4ab5-8857-c895e7fa863c '
                        '([41267d90-8023-441b-acbd-b0404c2f41de])'},
  'name': '41267d90-8023-441b-acbd-b0404c2f41de-2021-2-19-1613772892724_edges',
  'organization_id': '92161827-b871-4ab5-8857-c895e7fa863c'},
 '43edad96-1f5c-42b9-9b4f-8cc14e0cd9dd': {'data_set_type': 'EntitySet',
  'external_id': '43edad96-1f5c-42b9-9b4f-8cc14e0cd9dd',
  'id': '43edad96-1f5c-42b9-9b4f-8cc14e0cd9dd',
  'metadata': {'contacts': [],
               'description': 'This is a

In [4]:
dataSetMetadataAPI.get_data_set_metadata("6cd52949-88af-4179-a8f1-93336d5d9a71")

{'data_set_type': 'ExternalTable',
 'external_id': '1778717381',
 'id': '6cd52949-88af-4179-a8f1-93336d5d9a71',
 'metadata': {'contacts': ['yue@openlattice.com'],
              'description': '',
              'flags': [],
              'metadata': [],
              'title': 'Hello'},
 'name': 'new_test_data',
 'organization_id': '92161827-b871-4ab5-8857-c895e7fa863c'}

In [5]:
dataSetMetadataAPI.get_data_set_column_metadata(
    '6cd52949-88af-4179-a8f1-93336d5d9a71', 
    '82333f39-25f7-4d85-87c5-24e2d79e05d6'
)

{'acl_key': ['6cd52949-88af-4179-a8f1-93336d5d9a71',
             '82333f39-25f7-4d85-87c5-24e2d79e05d6'],
 'data_set_id': '6cd52949-88af-4179-a8f1-93336d5d9a71',
 'data_type': 'BIGINT',
 'id': '82333f39-25f7-4d85-87c5-24e2d79e05d6',
 'metadata': {'contacts': ['yue@openlattice.com'],
              'description': '',
              'flags': [],
              'metadata': [],
              'title': 'Hello'},
 'name': 'b',
 'organization_id': '92161827-b871-4ab5-8857-c895e7fa863c'}

## To get multiple results for metadata...

If you want to find metadata for multiple values, use `get_data_sets_metadata` and `get_data_set_columns_metadata`. Instead of passing in UUID directly, set `request_body` to a list of UUIDs. Both take the UUIDs of datasets (`get_data_set_columns_metadata` will return all of the column metadata associated with the datasets you passed in). 

In [6]:
dataSetMetadataAPI.get_data_sets_metadata(request_body = ['6cd52949-88af-4179-a8f1-93336d5d9a71'])

{'6cd52949-88af-4179-a8f1-93336d5d9a71': {'data_set_type': 'ExternalTable',
  'external_id': '1778717381',
  'id': '6cd52949-88af-4179-a8f1-93336d5d9a71',
  'metadata': {'contacts': ['yue@openlattice.com'],
               'description': '',
               'flags': [],
               'metadata': [],
               'title': 'Hello'},
  'name': 'new_test_data',
  'organization_id': '92161827-b871-4ab5-8857-c895e7fa863c'}}

In [7]:
dataSetMetadataAPI.get_data_set_columns_metadata(request_body = ['6cd52949-88af-4179-a8f1-93336d5d9a71'])

{'6cd52949-88af-4179-a8f1-93336d5d9a71': [{'acl_key': ['6cd52949-88af-4179-a8f1-93336d5d9a71',
               '82333f39-25f7-4d85-87c5-24e2d79e05d6'],
   'data_set_id': '6cd52949-88af-4179-a8f1-93336d5d9a71',
   'data_type': 'BIGINT',
   'id': '82333f39-25f7-4d85-87c5-24e2d79e05d6',
   'metadata': {'contacts': ['yue@openlattice.com'],
                'description': '',
                'flags': [],
                'metadata': [],
                'title': 'Hello'},
   'name': 'b',
   'organization_id': '92161827-b871-4ab5-8857-c895e7fa863c'},
  {'acl_key': ['6cd52949-88af-4179-a8f1-93336d5d9a71',
               '568a1e84-a5bc-45f5-8ed9-4675813a2adb'],
   'data_set_id': '6cd52949-88af-4179-a8f1-93336d5d9a71',
   'data_type': 'BIGINT',
   'id': '568a1e84-a5bc-45f5-8ed9-4675813a2adb',
   'metadata': {'contacts': [],
                'description': '',
                'flags': [],
                'metadata': [],
                'title': 'a'},
   'name': 'a',
   'organization_id': '92161827-b8

## To update metadata...

Pass in the metadata object with the appropriate metadata information. 

Pass in data set UUID to `update_data_set_metadata`, and if also updating column, pass in column UUID to `update_data_set_column_metadata`. 

In [12]:
metadata = openlattice.SecurableObjectMetadataUpdate(title = "Bye", contacts = ['yue@openlattice.com'])
dataSetMetadataAPI.update_data_set_metadata(
    '6cd52949-88af-4179-a8f1-93336d5d9a71', 
    securable_object_metadata_update = metadata
)

In [13]:
dataSetMetadataAPI.get_data_sets_metadata(request_body = ['6cd52949-88af-4179-a8f1-93336d5d9a71'])

{'6cd52949-88af-4179-a8f1-93336d5d9a71': {'data_set_type': 'ExternalTable',
  'external_id': '1778717381',
  'id': '6cd52949-88af-4179-a8f1-93336d5d9a71',
  'metadata': {'contacts': ['yue@openlattice.com'],
               'description': '',
               'flags': [],
               'metadata': [],
               'title': 'Bye'},
  'name': 'new_test_data',
  'organization_id': '92161827-b871-4ab5-8857-c895e7fa863c'}}

In [14]:
dataSetMetadataAPI.update_data_set_column_metadata(
    '6cd52949-88af-4179-a8f1-93336d5d9a71', 
    '82333f39-25f7-4d85-87c5-24e2d79e05d6', 
    securable_object_metadata_update = metadata
)

In [15]:
dataSetMetadataAPI.get_data_set_column_metadata(
    '6cd52949-88af-4179-a8f1-93336d5d9a71', 
    '82333f39-25f7-4d85-87c5-24e2d79e05d6'
)

{'acl_key': ['6cd52949-88af-4179-a8f1-93336d5d9a71',
             '82333f39-25f7-4d85-87c5-24e2d79e05d6'],
 'data_set_id': '6cd52949-88af-4179-a8f1-93336d5d9a71',
 'data_type': 'BIGINT',
 'id': '82333f39-25f7-4d85-87c5-24e2d79e05d6',
 'metadata': {'contacts': ['yue@openlattice.com'],
              'description': '',
              'flags': [],
              'metadata': [],
              'title': 'Bye'},
 'name': 'b',
 'organization_id': '92161827-b871-4ab5-8857-c895e7fa863c'}